# Import

In [7]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
from time import time

from tensorflow.keras.datasets import cifar10

# DenseNet 121

In [8]:
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model

In [9]:
input_shape = 32, 32, 3
n_classes = 10

In [13]:
model = densenet(input_shape,n_classes)
model.compile( loss=tf.keras.losses.sparse_categorical_crossentropy ,optimizer=tf.keras.optimizers.Adam( lr=0.0001 ) ,metrics=[ 'acc' ])
model.summary()
print(len(model.layers))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_242 (Conv2D)             (None, 16, 16, 64)   9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 8, 8, 64)     0           conv2d_242[0][0]                 
__________________________________________________________________________________________________
batch_normalization_240 (BatchN (None, 8, 8, 64)     256         max_pooling2d_2[0][0]            
____________________________________________________________________________________________

# Data

In [11]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

# normalize to range 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

val_x = train_x[:5000]
val_y = train_y[:5000]

170500096/170498071 [==============================] - 2s 0us/step


# Train model

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', verbose=1)
list_cb = [es]

In [14]:

start = time()
model.fit( train_x , train_y , epochs=50, batch_size=1024, validation_split=0.3, callbacks=list_cb)
training_time = time()-start

Epoch 1/5
782/782 [==============================] - 792s 997ms/step - loss: 1.7355 - acc: 0.3655 - val_loss: 1.4769 - val_acc: 0.4656
Epoch 2/5
782/782 [==============================] - 785s 1s/step - loss: 1.2401 - acc: 0.5513 - val_loss: 1.1751 - val_acc: 0.5718
Epoch 3/5
782/782 [==============================] - 786s 1s/step - loss: 1.0366 - acc: 0.6288 - val_loss: 1.0631 - val_acc: 0.6194
Epoch 4/5
782/782 [==============================] - 785s 1s/step - loss: 0.8749 - acc: 0.6878 - val_loss: 1.0099 - val_acc: 0.6414
Epoch 5/5
782/782 [==============================] - 785s 1s/step - loss: 0.7429 - acc: 0.7361 - val_loss: 1.0449 - val_acc: 0.6384


# Evaluate

In [15]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 34s 107ms/step - loss: 1.4190 - acc: 0.5551


[1.4190149307250977, 0.5551000237464905]

In [ ]:
model.evaluate(train_x, train_y)

In [ ]:
training_time